In [110]:
import scanpy as sc
import pandas as pd 
import numpy as np
import mygene 
from gprofiler import GProfiler


In [149]:
df = pd.read_csv('GSE167823_GEO_supporting_processed_data_file_raw_count_matrix.csv.gz',index_col=0).transpose()


In [117]:
mg = mygene.MyGeneInfo()

gene_info = mg.querymany(df.columns.tolist(), scopes='ensembl.gene', fields='symbol,name', species='human')


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
29 input query terms found dup hits:	[('ENSG00000228044', 2), ('ENSG00000226506', 2), ('ENSG00000261600', 2), ('ENSG00000234162', 2), ('E
1921 input query terms found no hit:	['ENSG00000238009', 'ENSG00000241670', 'ENSG00000236743', 'ENSG00000278566', 'ENSG00000224813', 'ENS


In [118]:
gene_info = pd.DataFrame(gene_info)


In [119]:
gene_list_full=df.columns.tolist()
gene_info_missing=gene_info[gene_info['notfound']==True]

In [120]:
additional_info=pd.read_csv("lncipedia_5_2_ensembl_92_genes.txt",sep="\t")
additional_info['query'] = additional_info['ensemblGeneID'].str.replace(r'\..*', '', regex=True)
additional_info=additional_info[additional_info['query'].isin(gene_info_missing['query'].tolist())]

In [121]:
gene_info = gene_info.merge(additional_info, on='query', how='left')


In [122]:
gene_info['symbol'] = gene_info['symbol'].combine_first(gene_info['lncipediaGeneID'])


In [123]:
gene_info['symbol'] = gene_info['symbol'].fillna(gene_info['query'])

In [125]:
gene_info = gene_info.drop_duplicates(subset='query', keep=False)

In [127]:
mapping_dict = dict(zip(gene_info['query'].tolist(), gene_info['symbol'].tolist()))


In [150]:
def are_columns_identical(col1, col2):
    return df[col1].equals(df[col2])

new_columns = []
for col in df.columns:
    new_name = mapping_dict.get(col, col)
    
    if new_name in new_columns:
        duplicate_found = any(are_columns_identical(existing_col, col) for existing_col in df.columns[:new_columns.index(new_name)])
        if duplicate_found:
            new_name = col

    new_columns.append(new_name)

df.columns = new_columns

In [151]:
adata = sc.AnnData(df)

/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [162]:
adata.write_h5ad("adata.h5ad")